In [1]:
from helpers.data import get_dataloaders
from helpers.train import TrainingManager
from helpers.loss_accuracy import accuracy
import models.resnet
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

In [2]:
trainloader, testloader = get_dataloaders('cifar10', 32)

Files already downloaded and verified


In [3]:
device = torch.device('cuda')

In [4]:
model = models.resnet.resnet32()
model = model.to(device)
optimizer = Adam(model.parameters(), 2e-4)

In [5]:
trial_name = "resnet32_adam_2e-4"

In [6]:
tm = TrainingManager(trial_name, load=False, is_trial=False)

In [7]:
tm.train(model, optimizer, 
         trainloader, testloader, 
         CrossEntropyLoss(), CrossEntropyLoss(), 
         accuracy, accuracy, device=device, no_iterations=100000)

  0%|          | 0/100000 [00:00<?, ?it/s]

Start training trial: resnet32_adam_2e-4


{tr_loss: 1.09544, tr_acc: 0.62031, te_loss: 0.98222, te_acc: 0.64219}:   6%|▌         | 5755/100000 [04:20<1:11:09, 22.08it/s] 


KeyboardInterrupt: 